<a href="https://colab.research.google.com/github/dw3101/ELIZA/blob/main/ELIZA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
ELIZA Chatbot Implementation
A recreation of the classic 1960s psychotherapist chatbot using pattern matching
and substitution methodologies.

Author: Implementation based on Joseph Weizenbaum's original ELIZA
"""

import re
import random

class Eliza:
    def __init__(self):
        # Reflection dictionary - converts "I" to "you", etc.
        self.reflections = {
            "am": "are",
            "was": "were",
            "i": "you",
            "i'd": "you would",
            "i've": "you have",
            "i'll": "you will",
            "my": "your",
            "are": "am",
            "you've": "I have",
            "you'll": "I will",
            "your": "my",
            "yours": "mine",
            "you": "me",
            "me": "you",
            "myself": "yourself",
            "yourself": "myself"
        }

        # Pattern-response pairs
        self.patterns = [
            # Quit patterns
            (r'quit|exit|bye|goodbye',
             ["Goodbye. It was nice talking to you.",
              "Thank you for talking with me. Goodbye.",
              "Goodbye. Take care."]),

            # Greetings
            (r'hello|hi|hey',
             ["Hello. How are you feeling today?",
              "Hi there. What brings you here today?",
              "Hello. Please tell me what's on your mind."]),

            # Positive emotions - specific patterns
            (r'i am (so |very |really )?(happy|excited|joyful|thrilled|great|amazing|wonderful|fantastic)',
             ["That's wonderful! What's making you feel %2?",
              "I'm glad to hear that! Tell me more about what's going well.",
              "That's great! What happened?",
              "How nice! What brings you this happiness?"]),

            (r'i feel (so |very |really )?(happy|excited|joyful|thrilled|great|amazing|wonderful|fantastic)',
             ["That sounds wonderful! What's bringing you joy?",
              "I'm happy to hear that. What's going on?",
              "That's great! Tell me more.",
              "What's making you feel this way?"]),

            (r'i\'m (so |very |really )?(happy|excited|joyful|thrilled|great|amazing|wonderful|fantastic)',
             ["That's wonderful! What happened?",
              "I'm glad to hear it! Tell me more.",
              "That sounds great! What's going on?",
              "How nice! What's making you feel this way?"]),

            # Negative emotions
            (r'i am (so |very |really )?(sad|depressed|unhappy|miserable|awful|terrible|angry|frustrated|anxious|worried)',
             ["I'm sorry you're feeling %2. What's been happening?",
              "That must be difficult. Can you tell me more?",
              "What's making you feel %2?",
              "How long have you been feeling this way?"]),

            (r'i feel (so |very |really )?(sad|depressed|unhappy|miserable|awful|terrible|angry|frustrated|anxious|worried)',
             ["I'm sorry to hear that. What's going on?",
              "That sounds difficult. Tell me more.",
              "What's making you feel %2?",
              "When did you start feeling this way?"]),

            (r'i\'m (so |very |really )?(sad|depressed|unhappy|miserable|awful|terrible|angry|frustrated|anxious|worried)',
             ["I'm sorry you're feeling %2. What happened?",
              "That must be tough. Can you tell me more?",
              "What's been troubling you?",
              "How long have you felt this way?"]),

            # Single word emotional responses
            (r'^(happy|excited|joyful|thrilled|great|amazing|wonderful|fantastic)$',
             ["That's great to hear! What's making you feel that way?",
              "Wonderful! Tell me more.",
              "I'm glad! What happened?",
              "That's nice! What's going on?"]),

            (r'^(sad|depressed|unhappy|miserable|awful|terrible|angry|frustrated|anxious|worried|bad)$',
             ["I'm sorry to hear that. What's wrong?",
              "That's tough. What's been happening?",
              "What's troubling you?",
              "Can you tell me more about that?"]),

            (r'^(good|fine|okay|ok|alright|well)$',
             ["That's good. What would you like to talk about?",
              "I'm glad. What's on your mind?",
              "Good to hear. Is there anything specific you'd like to discuss?",
              "That's nice. What brings you here today?"]),

            # Life/Things are good or bad
            (r'(life|things|everything) (is|are) (so |very |really )?(good|great|wonderful|amazing|fantastic)',
             ["That's wonderful! What's been going well?",
              "I'm happy to hear that! What's making life good?",
              "That's great! Tell me more about it.",
              "How nice! What's contributing to this?"]),

            (r'(life|things|everything) (is|are) (so |very |really )?(bad|hard|difficult|tough|awful|terrible)',
             ["I'm sorry things are tough. What's been difficult?",
              "That sounds hard. What's been happening?",
              "What's making things difficult for you?",
              "Can you tell me more about what you're going through?"]),

            # I need/want patterns
            (r'i need (.*)',
             ["Why do you need %1?",
              "Would it help you to get %1?",
              "What would it mean to you to get %1?"]),

            (r'i want (.*)',
             ["What would it mean if you got %1?",
              "Why do you want %1?",
              "What would change if you got %1?"]),

            # Why questions about ELIZA
            (r'why (did|do) you ask',
             ["I'm trying to understand you better.",
              "Does it bother you that I asked?",
              "I'm here to listen and help you explore your thoughts.",
              "What made you curious about why I asked?"]),

            (r'(because|cause|cuz|bc) you (asked|said|told) (.*)',
             ["I see. And how do you feel about %3?",
              "What else can you tell me about that?",
              "Tell me more.",
              "Go on."]),

            # Why questions
            (r'why don\'?t you (.*)',
             ["Would you like me to %1?",
              "Perhaps you'd like me to %1?",
              "Should I %1?"]),

            (r'why can\'?t i (.*)',
             ["Do you think you should be able to %1?",
              "What do you think is stopping you?",
              "What would it take for you to %1?"]),

            # I can't patterns
            (r'i can\'?t (.*)',
             ["How do you know you can't %1?",
              "What's preventing you from %1?",
              "Have you tried to %1?"]),

            # I am/I'm patterns - neutral/other states
            (r'i am (.*)',
             ["How long have you been %1?",
              "How do you feel about being %1?",
              "What made you %1?"]),

            (r'i\'m (.*)',
             ["How do you feel about being %1?",
              "What made you %1?",
              "Tell me more about being %1."]),

            # Incomplete thoughts
            (r'^i think$',
             ["What do you think?",
              "Go on...",
              "You were saying?"]),

            (r'^i feel$',
             ["What do you feel?",
              "Tell me more.",
              "How do you feel?"]),

            # I feel patterns
            (r'i feel (.*)',
             ["Tell me more about feeling %1.",
              "Do you often feel %1?",
              "What makes you feel %1?"]),

            # I think patterns
            (r'i think (.*)',
             ["Why do you think %1?",
              "Do you really think so?",
              "What makes you think %1?"]),

            # I like/love patterns
            (r'i (like|love|enjoy) (.*)',
             ["What do you %1 about %2?",
              "What makes %2 special to you?",
              "How does %2 make you feel?"]),

            (r'it makes me (.*)',
             ["Why does it make you %1?",
              "What is it about it that makes you %1?",
              "Tell me more about that."]),

            # Just did something
            (r'i just (.*)',
             ["How was %1?",
              "How did %1 make you feel?",
              "Tell me about %1."]),

            # Patterns about others
            (r'(.*) (friend|friends) (.*)',
             ["Tell me more about your friends.",
              "How do your friends make you feel?"]),

            (r'(.*) mother(.*)',
             ["Tell me more about your mother.",
              "What was your relationship with your mother like?"]),

            (r'(.*) father(.*)',
             ["Tell me more about your father.",
              "How did your father make you feel?"]),

            (r'(.*) family (.*)',
             ["Tell me more about your family.",
              "How does your family make you feel?"]),

            # Yes/No responses
            (r'^yes$',
             ["I see.",
              "Tell me more.",
              "And what does that mean to you?"]),

            (r'^no$',
             ["Why not?",
              "Can you elaborate?",
              "Tell me more."]),

            # Dreams
            (r'(.*) dream (.*)',
             ["What does that dream suggest to you?",
              "Do you dream often?",
              "Tell me more about your dreams."]),

            # Remember patterns
            (r'i remember (.*)',
             ["Do you often think of %1?",
              "What else do you remember?",
              "Why do you remember %1 now?"]),

            # Always/everyone patterns
            (r'(.*) always (.*)',
             ["Can you think of a specific example?",
              "Always?",
              "What makes you say always?"]),

            (r'(.*) (everyone|everybody) (.*)',
             ["Surely not everyone?",
              "Can you think of anyone in particular?"]),

            # Because patterns
            (r'because (.*)',
             ["Is that the real reason?",
              "What other reasons might there be?",
              "Tell me more about %1."]),

            # Time-related responses
            (r'since (yesterday|today|this morning|this afternoon|last night|last week|.*)',
             ["What happened %1?",
              "Tell me about %1.",
              "What was going on %1?"]),

            (r'for (a while|a long time|.*)',
             ["How long exactly?",
              "When did it start?",
              "What's been happening?"]),

            # Catch-all patterns
            (r'(.*)',
             ["Tell me more.",
              "Can you elaborate?",
              "Go on.",
              "I'm listening.",
              "What else?",
              "And how does that make you feel?"])
        ]

        self.memory = []

    def reflect(self, text):
        """
        Reflect pronouns and verb forms (e.g., "I am" -> "you are")
        """
        words = text.lower().split()
        reflected = []
        for word in words:
            reflected.append(self.reflections.get(word, word))
        return ' '.join(reflected)

    def analyze(self, statement):
        """
        Analyze the user's statement and generate a response
        """
        statement = statement.lower().strip()

        # Try to match patterns
        for pattern, responses in self.patterns:
            match = re.match(pattern, statement)
            if match:
                response = random.choice(responses)

                # Replace placeholders with captured groups (reflected)
                if '%' in response:
                    for i, group in enumerate(match.groups(), start=1):
                        placeholder = f'%{i}'
                        if placeholder in response:
                            reflected = self.reflect(group)
                            response = response.replace(placeholder, reflected)

                # Check if this is a quit command
                if pattern.startswith('quit|exit'):
                    return response, True

                return response, False

        # If no pattern matched
        return "Please tell me more.", False

    def run(self):
        """
        Main conversation loop
        """
        print("ELIZA: Hello. I am ELIZA, your psychotherapist.")
        print("ELIZA: Please tell me what's been on your mind.")
        print("ELIZA: (Type 'quit' to exit)\n")

        while True:
            try:
                user_input = input("You: ").strip()

                if not user_input:
                    print("ELIZA: Please say something.\n")
                    continue

                response, should_quit = self.analyze(user_input)
                print(f"ELIZA: {response}\n")

                if should_quit:
                    break

            except (KeyboardInterrupt, EOFError):
                print("\nELIZA: Goodbye.")
                break


# Main execution
if __name__ == "__main__":
    eliza = Eliza()
    eliza.run()

ELIZA: Hello. I am ELIZA, your psychotherapist.
ELIZA: Please tell me what's been on your mind.
ELIZA: (Type 'quit' to exit)

ELIZA: Hi there. What brings you here today?

ELIZA: What else?

